In [5]:
import numpy as np  #gereki oan kütüphaneleri (paketleri) içe aktarıyoruz
import os 
import cv2

CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",  #sınıfları yani etiketleri tanımlıyoruz
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]

COLORS = np.random.uniform(0,255,size=(len(CLASSES),3)) #sınıf sayısı kadar renk üretiyoruz

net = cv2.dnn.readNetFromCaffe("MobileNetSSD_deploy.prototxt.txt", "MobileNetSSD_deploy.caffemodel")  #önceden eğitilmiş MobileNet SSD modelini yükleriz

files = os.listdir()  #mevcut dosya yolu 
img_path_list = []  #jpg uzantılı dosyaları bu liste içinde tutacağız
for f in files: #dosyaların içinde geziniyoruz
    if f.endswith(".jpg"): #jpg uzantılı olan dosyaları img_path_list listesine ekliyoruz
        img_path_list.append(f)
        
for i in img_path_list: #listenin içinde geziniyoruz
    
    image = cv2.imread(i) #resimleri okuyourz
    (h,w) = image.shape[:2] #resimlerin height ve width değerlerini alıyoruz
    blob = cv2.dnn.blobFromImage(cv2.resize(image,(300, 300)), 0.007843,(300, 300), 127.5)  #ssd 300x300 kabul ediyor diğerleri ise modelle ilgili parametreler (ortalama değer çıkarımı yapılır)
    
    net.setInput(blob) 
    detections = net.forward() #nesne algılama sonuçları detections değişkenine atılır 
    
    for j in np.arange(0, detections.shape[2]):  #her bir algılanan nesne için
        
        confidence = detections[0,0,j,2]
        
        if confidence > 0.38:  #eğer ki olasılık(tahmin) değeri 0.38'den buyukse
            
            idx = int(detections[0,0,j,1]) #algılanan nesnenin sınıf indexi hesaplanır
            box = detections[0,0,j,3:7]*np.array([w,h,w,h]) #x ,y ,width ve height değerleri hesaplanır
            (startX, startY, endX, endY) = box.astype("int")  #box değişkenindeki koordinatlar int'e dönüştürdükten sonra (startX, startY, endX, endY) değişkenlerine eşitlenir
            
            label = "{}: {}".format(CLASSES[idx], confidence) #tespit edilen nesnelerin sınıfı ve tahmin değeri label değişkeninde
            cv2.rectangle(image, (startX, startY), (endX, endY), COLORS[idx],2)  #bulunan koordinatlarda image üzerine nesnelerin çevresine dikdörtgen(sınırlayıcı kutu) çizeriz
            y = startY - 16 if startY -16 >15 else startY + 16 #absürt bir konumda bilgilendirme olmaması adına
            cv2.putText(image, label, (startX,y),cv2.FONT_HERSHEY_SIMPLEX, 0.5,COLORS[idx],2) #image'imizin üstüne labeldeki verileri yazdırırız 
            
    cv2.imshow("ssd",image) #ve en son ssd modeli ile tespit edilmiş nesneleri imshowlarız
    if cv2.waitKey(0) == ord('q'): #eğer ki 'q' ya basılmışsa bir sonraki resimden devam et
        continue
cv2.destroyAllWindows() #tüm pencereleri kapat